<h1>Обработка файлов с данными о ЗНО<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Словарь-и-тестовый-прогон" data-toc-modified-id="Словарь-и-тестовый-прогон-1">Словарь и тестовый прогон</a></span></li><li><span><a href="#Полная-обработка-датасетов" data-toc-modified-id="Полная-обработка-датасетов-2">Полная обработка датасетов</a></span></li><li><span><a href="#Геополигоны" data-toc-modified-id="Геополигоны-3">Геополигоны</a></span></li><li><span><a href="#Гипотеза" data-toc-modified-id="Гипотеза-4">Гипотеза</a></span></li><li><span><a href="#Таблица-57-(доп.-инфа)" data-toc-modified-id="Таблица-57-(доп.-инфа)-5">Таблица 57 (доп. инфа)</a></span></li><li><span><a href="#Доп.-обработка-датасета.-Правки-от-заказчика" data-toc-modified-id="Доп.-обработка-датасета.-Правки-от-заказчика-6">Доп. обработка датасета. Правки от заказчика</a></span></li></ul></div>

## Словарь и тестовый прогон

In [1]:
import pandas as pd
import warnings
import random
warnings.filterwarnings('ignore')

    Сначала сделаем словарь с локализациями опухолей и кодом МКБ 10

In [2]:
codes_df = pd.read_excel(f'datasets\\2021_Состояние_онко_помощи_в_РФ_Таблицы_54_и_57-85\\2021_Таблица_054_Состояние_онко_помощи_в_РФ.xlsx', header=None)
codes_df = codes_df.iloc[2:,0:2].reset_index(drop=True)

header = codes_df.iloc[0]
codes_df = codes_df[2:].reset_index(drop=True)
codes_df.columns = header

codes_dict = codes_df.set_index('Локализация опухоли').T.to_dict('list')

    Теперь тестовый код для одного датасета

In [3]:
num = 59
df = pd.read_excel(f'datasets\\2021_Состояние_онко_помощи_в_РФ_Таблицы_54_и_57-85\\2021_Таблица_0{num}_Состояние_онко_помощи_в_РФ.xlsx')

In [4]:
form = df.columns[0].split("\n")[2].split(" Т")[0]
form = form[:form.rfind("(") - 1]
display(form)
display(codes_dict[form][0])

df.columns = ["Область", "Леч_законч", "Леч_зак_%_от_нов", "Леч_продолж", "Леч_прод_%_от_нов", 
                "Метод_хир", "Метод_луч", "Метод_лек", "Метод_ком", "Метод_хим"]
df = df.loc[3:].reset_index(drop=True)

'У детей в возрасте 0-14 лет'

'С00-96'

In [5]:
df.insert(loc=0, column='Локализация опухоли', value=form)
df.insert(loc=0, column='Код МКБ 10', value=codes_dict[form][0])

In [6]:
fed_okr_series = []
drop_list = []
fed_okr = None
for i, col in df.iterrows():
    if "ФО" in col['Область']:
        fed_okr = col['Область']
        drop_list.append(i)
    fed_okr_series.append(fed_okr)

In [7]:
df.insert(loc=2, column='ФО', value=fed_okr_series)
df=df.drop(index=drop_list).reset_index(drop=True)

## Полная обработка датасетов

    И наконец фулл цикл обработки однотипных датасетов

In [8]:
full_df = None
for num in range(58,85+1):
    df = pd.read_excel(
        f'datasets\\2021_Состояние_онко_помощи_в_РФ_Таблицы_54_и_57-85\\2021_Таблица_0{num}_Состояние_онко_помощи_в_РФ.xlsx')
    #Для начала вынимаем локализацию опухоли и код МКБ 10
    form = df.columns[0].split("\n")[2].split(" Т")[0]
    form = form[:form.rfind("(") - 1]
    #display(codes_dict[form][0])

    #Обрабатываем датасет, добавляем колонки с локализацией и кодом
    df.columns = ["Область", "Леч_законч", "Леч_зак_%_от_нов", "Леч_продолж", "Леч_прод_%_от_нов", 
            "Метод_хир", "Метод_луч", "Метод_лек", "Метод_ком", "Метод_хим"]
    df = df.loc[3:].reset_index(drop=True)
    df.insert(loc=0, column='Код МКБ 10', value=codes_dict[form][0])
    df.insert(loc=0, column='Локализация опухоли', value=form)
    #display(df.head(1))

    #Вынимаем ФО и добавляем в отдельную колонку
    fed_okr_series = []
    drop_list = []
    fed_okr = None
    for i, col in df.iterrows():
        if "ФО" in col['Область']:
            fed_okr = col['Область']
            drop_list.append(i)
        fed_okr_series.append(fed_okr)

    df.insert(loc=2, column='ФО', value=fed_okr_series)
    df=df.drop(index=drop_list).reset_index(drop=True)

    #Добавляем в общий датасет
    full_df = pd.concat([full_df, df]).reset_index(drop=True)
    
display(full_df)

,Локализация опухоли,Код МКБ 10,ФО,Область,Леч_законч,Леч_зак_%_от_нов,Леч_продолж,Леч_прод_%_от_нов,Метод_хир,Метод_луч,Метод_лек,Метод_ком,Метод_хим
0,Злокачественные новообразования - всего,С00-96,ЦЕНТРАЛЬНЫЙ ФО,Белгородская область,3952,57.1,1418,20.5,53.6,12.7,2.9,27.9,2.9
1,Злокачественные новообразования - всего,С00-96,ЦЕНТРАЛЬНЫЙ ФО,Брянская область,2673,46.9,1236,21.7,54,2.3,7.6,34.3,1.8
2,Злокачественные новообразования - всего,С00-96,ЦЕНТРАЛЬНЫЙ ФО,Владимирская область,2858,51.9,808,14.7,59,2.4,0,37.5,1.1
3,Злокачественные новообразования - всего,С00-96,ЦЕНТРАЛЬНЫЙ ФО,Воронежская область,5305,58.8,1635,18.1,53.9,9.4,2.3,32.5,1.8
4,Злокачественные новообразования - всего,С00-96,ЦЕНТРАЛЬНЫЙ ФО,Ивановская область,2289,49.9,553,12.1,58,10.6,1.8,28,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2363,Лейкемии,С91-95,ДАЛЬНЕВОСТОЧНЫЙ ФО,Забайкальский край,26,44.8,12,20.7,0,0,100,0,0
2364,Лейкемии,С91-95,ДАЛЬНЕВОСТОЧНЫЙ ФО,Чукотский авт.округ,0,0,1,100,0,0,0,0,0
2365,Лейкемии,С91-95,ДАЛЬНЕВОСТОЧНЫЙ ФО,Республика Бурятия,20,62.5,12,37.5,0,0,95,5,0
2366,Лейкемии,С91-95,ДАЛЬНЕВОСТОЧНЫЙ ФО,Республика Саха (Якутия),31,58.5,9,17,0,0,100,0,0


    Сменим типы данных в колонках получившегося датасета

In [9]:
list_cols = full_df.columns.tolist()
list_cols = list_cols[4:]
del list_cols[0]
del list_cols[1]

In [10]:
full_df['Леч_законч'] = full_df['Леч_законч'].astype(int)
full_df['Леч_продолж'] = full_df['Леч_продолж'].astype(int)

for col in list_cols:
    full_df[col] = full_df[col].astype(float)

full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2368 entries, 0 to 2367
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Локализация опухоли  2368 non-null   object 
 1   Код МКБ 10           2368 non-null   object 
 2   ФО                   2368 non-null   object 
 3   Область              2368 non-null   object 
 4   Леч_законч           2368 non-null   int32  
 5   Леч_зак_%_от_нов     2368 non-null   float64
 6   Леч_продолж          2368 non-null   int32  
 7   Леч_прод_%_от_нов    2368 non-null   float64
 8   Метод_хир            2368 non-null   float64
 9   Метод_луч            2368 non-null   float64
 10  Метод_лек            2368 non-null   float64
 11  Метод_ком            2368 non-null   float64
 12  Метод_хим            2368 non-null   float64
dtypes: float64(7), int32(2), object(4)
memory usage: 222.1+ KB


## Геополигоны

    Для отображения на карте придется покопаться с геополигонами. Датасет взял по ссылке от DataLens

In [11]:
geo_regions = pd.read_csv('datasets\\ru_regions.csv', sep = ';')
geo_data = geo_regions[geo_regions['coords_type'] == 'ru_regions_poly'][['region', 'coords']]
geo_data['region'] = [poly.replace(" (Югра)", "").replace(" - Алания", "").replace("республика", "Республика") for poly in geo_data['region']]

display(geo_data)

geo_data.to_csv('datasets\\geo_data.csv', sep=';', encoding='utf-8', index=False)

,region,coords
0,Ростовская область,"[[[50.21447689972731,41.3502545996028],[50.199..."
1,Рязанская область,"[[[55.36424399994077,40.210444700223796],[55.3..."
2,Сахалинская область,"[[[46.4051481123399,142.82388437012776],[46.39..."
3,Самарская область,"[[[54.67642609980043,51.38997859973747],[54.66..."
4,Саратовская область,"[[[52.81440689976361,48.42863240015605],[52.76..."
...,...,...
80,Еврейская автономная область,"[[[49.48947989980819,132.1187526002292],[49.48..."
81,Пермский край,"[[[60.223260099883014,52.3278556000364],[60.22..."
82,Московская область,"[[[55.44980949998922,37.49815720008871],[55.44..."
83,Калининградская область,"[[[54.360271200058015,22.687794399726155],[54...."


In [12]:
full_df['Область'] = [name.replace("Респ.", "Республика")
         .replace("а.о.", "автономный округ")
         .replace("авт.округ", "автономный округ")
         .replace("авт. обл.", "автономная область")
         .replace("обл.(б/а.о)", "область")
         .replace("Город Cевастополь", "Севастополь") #"С" не русская! Опять(
         .replace("г. ", "") for name in full_df['Область']]

for i in full_df['Область']:
    if i not in geo_data['region'].unique():
        print(i)

    P.S. К сожалению карту пока не удалось настроить, но на будущее полигоны будут

In [13]:
#full_df_poly = full_df.merge(geo_data, left_on='Область', right_on='region', how="left").drop('region', 1)
#full_df_poly.to_csv('datasets\\med_poly.csv', sep=';', encoding='utf-8', index=False)

## Гипотеза

    Давайте кое-что проверим. Есть 2 категории - лечение закончено и лечение будет продолжено, верно?
    Но проценты не суммируются до 100, иногда даже близко. Однако они показывают правильный процент относительно друг друга и абсолютных значений. 
    Сейчас покажу на примере, для понимания.

In [14]:
num = random.randrange(len(full_df))
full_df.loc[[num]]

,Локализация опухоли,Код МКБ 10,ФО,Область,Леч_законч,Леч_зак_%_от_нов,Леч_продолж,Леч_прод_%_от_нов,Метод_хир,Метод_луч,Метод_лек,Метод_ком,Метод_хим
2132,Щитовидная железа,С73,СЕВЕРО-ЗАПАДНЫЙ ФО,Архангельская область,65,94.2,1,1.4,80.0,0.0,0.0,20.0,0.0


        Вот рандомная строка из датасета. Проверим следующее - из 'Леч_законч' и 'Леч_зак_%_от_нов' узнаем общее число ЗНО и дальше проверим через 'Леч_продолж' правильно ли указан процент в колонке 'Леч_прод_%_от_нов'

In [15]:
try:
    result = round(full_df['Леч_продолж'][num]/(full_df['Леч_законч'][num]/full_df['Леч_зак_%_от_нов'][num]*100)*100, 1)
except:
    result = "из-за нуля проверка ломается, попробуйте другое число (прогоните несколько ячеек заново)"

print('Леч_прод_%_от_нов -', full_df['Леч_прод_%_от_нов'][num])
print('Моя проверка -', result)

Леч_прод_%_от_нов - 1.4
Моя проверка - 1.4


    Как видите на рандомной проверке - моя гипотеза верна (можете проранить ещё несколько раз код). Ломается только на нулях. Из этого делаю вывод, что следует добавить 3 категорию - Другое. 
    Будет:
   - Число ЗНО, выявленных в отчетном году, радикальное лечение которых закончено в отчетном году;
   - Число ЗНО, выявленных в отчетном году, радикальное лечение которых будет продолжено (не закончено);
   - Другое.
   
    Поэтому на мой взгляд следует выделять 3 колонки, а не 2

## Таблица 57 (доп. инфа)

    Обработаем отдельно таблицу 57 с дополнительной информацией.

In [16]:
table_57 = pd.read_excel(f'datasets\\2021_Состояние_онко_помощи_в_РФ_Таблицы_54_и_57-85\\2021_Таблица_057_Состояние_онко_помощи_в_РФ.xlsx', header=None)

table_57.columns = ["Область", 
              "Отказы_от_лечения_абс_ЗНО", "Отказы_от_лечения_%_ЗНО", "Отказы_от_лечения_1-2_абс", "Отказы_от_лечения_1-2_%", 
              "Против_к_леч_абс_ЗНО", "Против_к_леч_%_ЗНО", "Против_к_леч_1-2_абс", "Против_к_леч_1-2_%", 
              "ЗНО_леч_амбул_абс", "ЗНО_леч_амбул_%", 
              "Законч_леч_лек_абс", "Законч_леч_лек_гемобл", "Законч_леч_луч_абс", 
              "Число_больных_лек_под_абс", "Число_больных_лек_пол_абс", "Число_больных_лек_пол_%", 
              "Число_больных_луч_под_абс", "Число_больных_луч_пол_абс", "Число_больных_луч_пол_%",
              "Число_больных_ком_под_абс", "Число_больных_ком_пол_абс", "Число_больных_ком_пол_%"]

table_57 = table_57.loc[6:].reset_index(drop=True)

In [17]:
fed_okr_series = []
drop_list = []
fed_okr = None
for i, col in table_57.iterrows():
    if "ФО" in col['Область']:
        fed_okr = col['Область']
        drop_list.append(i)
    fed_okr_series.append(fed_okr)

In [18]:
table_57.insert(loc=0, column='ФО', value=fed_okr_series)
table_57=table_57.drop(index=drop_list).reset_index(drop=True)
display(table_57)

,ФО,Область,Отказы_от_лечения_абс_ЗНО,Отказы_от_лечения_%_ЗНО,Отказы_от_лечения_1-2_абс,Отказы_от_лечения_1-2_%,Против_к_леч_абс_ЗНО,Против_к_леч_%_ЗНО,Против_к_леч_1-2_абс,Против_к_леч_1-2_%,...,Законч_леч_луч_абс,Число_больных_лек_под_абс,Число_больных_лек_пол_абс,Число_больных_лек_пол_%,Число_больных_луч_под_абс,Число_больных_луч_пол_абс,Число_больных_луч_пол_%,Число_больных_ком_под_абс,Число_больных_ком_пол_абс,Число_больных_ком_пол_%
0,ЦЕНТРАЛЬНЫЙ ФО,Белгородская область,90,1.3,30,33.3,132,1.9,22,16.7,...,1991,4631,4560,98.5,2044,1991,97.4,2437,2394,98.2
1,ЦЕНТРАЛЬНЫЙ ФО,Брянская область,20,0.4,9,45,97,1.7,12,12.4,...,1420,5716,5700,99.7,1438,1420,98.7,1587,1570,98.9
2,ЦЕНТРАЛЬНЫЙ ФО,Владимирская область,107,1.9,71,66.4,245,4.5,100,40.8,...,1257,7443,6949,93.4,1347,1257,93.3,3187,2270,71.2
3,ЦЕНТРАЛЬНЫЙ ФО,Воронежская область,111,1.2,41,36.9,335,3.7,82,24.5,...,2637,9896,9839,99.4,2653,2637,99.4,3912,3878,99.1
4,ЦЕНТРАЛЬНЫЙ ФО,Ивановская область,43,0.9,26,60.5,118,2.6,25,21.2,...,1462,5103,5051,99,1478,1462,98.9,3373,3341,99.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,ДАЛЬНЕВОСТОЧНЫЙ ФО,Забайкальский край,190,5.6,101,53.2,325,9.6,164,50.5,...,805,4263,3757,88.1,917,805,87.8,1982,1519,76.6
81,ДАЛЬНЕВОСТОЧНЫЙ ФО,Чукотский авт.округ,3,2.7,2,66.7,2,1.8,0,0,...,6,63,53,84.1,7,6,85.7,13,11,84.6
82,ДАЛЬНЕВОСТОЧНЫЙ ФО,Республика Бурятия,88,2.6,39,44.3,234,6.9,40,17.1,...,981,6269,6145,98,986,981,99.5,7458,7386,99
83,ДАЛЬНЕВОСТОЧНЫЙ ФО,Республика Саха (Якутия),22,1,9,40.9,187,8.1,37,19.8,...,412,1797,1624,90.4,414,412,99.5,573,530,92.5


In [19]:
table_57.to_csv('datasets\\table_57.csv', sep=';', encoding='utf-8', index=False)
table_57.to_excel('datasets\\table_57.xlsx', index=False)

## Доп. обработка датасета. Правки от заказчика

    Давайте удалим несколько пустых строк, которые будут только мешаться. В некоторых и так отсутствую данные, так что 0 или None не сильно различается. Но при построении чартов лучше наверное None. Так в таблице отчетливо видно, что и где отсутствует.

In [20]:
drop_list2 = full_df[(full_df['Леч_законч'] == 0) & (full_df['Леч_продолж'] == 0)].index.tolist()
full_df = full_df.drop(index=drop_list2).reset_index(drop=True)
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349 entries, 0 to 2348
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Локализация опухоли  2349 non-null   object 
 1   Код МКБ 10           2349 non-null   object 
 2   ФО                   2349 non-null   object 
 3   Область              2349 non-null   object 
 4   Леч_законч           2349 non-null   int32  
 5   Леч_зак_%_от_нов     2349 non-null   float64
 6   Леч_продолж          2349 non-null   int32  
 7   Леч_прод_%_от_нов    2349 non-null   float64
 8   Метод_хир            2349 non-null   float64
 9   Метод_луч            2349 non-null   float64
 10  Метод_лек            2349 non-null   float64
 11  Метод_ком            2349 non-null   float64
 12  Метод_хим            2349 non-null   float64
dtypes: float64(7), int32(2), object(4)
memory usage: 220.3+ KB


    Правки от заказчика.
    
    Сначала убираем категорию 'Злокачественные новообразования - всего' для удобства работы с дашбордом

In [21]:
full_df = full_df[full_df['Локализация опухоли'] != 'Злокачественные новообразования - всего'].reset_index(drop=True)

    Затем уберем строки с малым количеством данных.
    
    То есть до 5 зарегистрированных случаев (вкл).
    В случае с 'Кости и суставные хрящи' сделаю до 3 случаев (вкл), тк почти везде в данной категории данные очень малы.

In [22]:
drop_list3 = full_df[(full_df['Леч_законч']+full_df['Леч_продолж']<3) & (full_df['Локализация опухоли'] == 'Кости и суставные хрящи')].index.tolist()
drop_list4 = full_df[(full_df['Леч_законч']+full_df['Леч_продолж']<5) & (full_df['Локализация опухоли'] != 'Кости и суставные хрящи')].index.tolist()

full_df = full_df.drop(index=drop_list3)
full_df = full_df.drop(index=drop_list4).reset_index(drop=True)
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2147 entries, 0 to 2146
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Локализация опухоли  2147 non-null   object 
 1   Код МКБ 10           2147 non-null   object 
 2   ФО                   2147 non-null   object 
 3   Область              2147 non-null   object 
 4   Леч_законч           2147 non-null   int32  
 5   Леч_зак_%_от_нов     2147 non-null   float64
 6   Леч_продолж          2147 non-null   int32  
 7   Леч_прод_%_от_нов    2147 non-null   float64
 8   Метод_хир            2147 non-null   float64
 9   Метод_луч            2147 non-null   float64
 10  Метод_лек            2147 non-null   float64
 11  Метод_ком            2147 non-null   float64
 12  Метод_хим            2147 non-null   float64
dtypes: float64(7), int32(2), object(4)
memory usage: 201.4+ KB


    Меняем формат колонок с информацией о локализации опухолей и МКБ 10.
    
    Заказчик попросил формат "Лейкемии (С91-95)" вместо отдельных колонок.
    Код обработки переписывать лень, так что просто исправим тут за пару строчек (к тому же мне тот код нравится)

In [23]:
full_df['Локализация опухоли'] = full_df['Локализация опухоли']+' ('+full_df['Код МКБ 10']+')'
full_df = full_df.drop('Код МКБ 10', axis=1)

In [24]:
display(full_df)
full_df.info()

,Локализация опухоли,ФО,Область,Леч_законч,Леч_зак_%_от_нов,Леч_продолж,Леч_прод_%_от_нов,Метод_хир,Метод_луч,Метод_лек,Метод_ком,Метод_хим
0,У детей в возрасте 0-14 лет (С00-96),ЦЕНТРАЛЬНЫЙ ФО,Белгородская область,3,11.1,22,81.5,66.7,0.0,33.3,0.0,0.0
1,У детей в возрасте 0-14 лет (С00-96),ЦЕНТРАЛЬНЫЙ ФО,Брянская область,23,63.9,9,25.0,21.7,0.0,60.9,8.7,8.7
2,У детей в возрасте 0-14 лет (С00-96),ЦЕНТРАЛЬНЫЙ ФО,Владимирская область,2,14.3,12,85.7,50.0,0.0,0.0,50.0,0.0
3,У детей в возрасте 0-14 лет (С00-96),ЦЕНТРАЛЬНЫЙ ФО,Воронежская область,5,33.3,10,66.7,0.0,0.0,80.0,20.0,0.0
4,У детей в возрасте 0-14 лет (С00-96),ЦЕНТРАЛЬНЫЙ ФО,Ивановская область,9,42.9,2,9.5,22.2,0.0,55.6,22.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2142,Лейкемии (С91-95),ДАЛЬНЕВОСТОЧНЫЙ ФО,Магаданская область,5,62.5,0,0.0,0.0,0.0,100.0,0.0,0.0
2143,Лейкемии (С91-95),ДАЛЬНЕВОСТОЧНЫЙ ФО,Сахалинская область,0,0.0,20,74.1,0.0,0.0,0.0,0.0,0.0
2144,Лейкемии (С91-95),ДАЛЬНЕВОСТОЧНЫЙ ФО,Забайкальский край,26,44.8,12,20.7,0.0,0.0,100.0,0.0,0.0
2145,Лейкемии (С91-95),ДАЛЬНЕВОСТОЧНЫЙ ФО,Республика Бурятия,20,62.5,12,37.5,0.0,0.0,95.0,5.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2147 entries, 0 to 2146
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Локализация опухоли  2147 non-null   object 
 1   ФО                   2147 non-null   object 
 2   Область              2147 non-null   object 
 3   Леч_законч           2147 non-null   int32  
 4   Леч_зак_%_от_нов     2147 non-null   float64
 5   Леч_продолж          2147 non-null   int32  
 6   Леч_прод_%_от_нов    2147 non-null   float64
 7   Метод_хир            2147 non-null   float64
 8   Метод_луч            2147 non-null   float64
 9   Метод_лек            2147 non-null   float64
 10  Метод_ком            2147 non-null   float64
 11  Метод_хим            2147 non-null   float64
dtypes: float64(7), int32(2), object(3)
memory usage: 184.6+ KB


    И, наконец, выгрузка основного датасета

In [25]:
full_df.to_csv('datasets\\med_df.csv', sep=';', encoding='utf-8', index=False)
full_df.to_excel('datasets\\med_df.xlsx', index=False)